In [2]:
import tensorflow as tf
import os
import numpy as np
from math import pi

### 2.2 Representing tensors###
Tensorflow uses the infamous __tensors__ to represent matrices of any dimensionality. (E.g. a 3D matrix can be represented
as a _rank 3_ tensor in tensorflow). <br>
Tensors cann be created from different sources such as _Python lists, numpy-arrays, tensorflow-constants, etc._ by using the: <br> `tf.convert_to_tensor([source])` <br>
function. `Notice that tf.constant()` also returns a Tensor object:

In [9]:
py_list = [[1.0, 2.0],
           [3.0, 4.0]]

numpy_array = np.array([[1.0, 2.0],
                        [3.0, 4.0]], dtype=np.float32)

tf_constant = tf.constant([[1.0, 2.0],
                           [3.0, 4.0]])

tensor1 = tf.convert_to_tensor(py_list, dtype=tf.float32)
tensor2 = tf.convert_to_tensor(numpy_array, dtype=tf.float32)
tensor3 = tf.convert_to_tensor(tf_constant, dtype=tf.float32)

print(type(py_list))
print(type(numpy_array))
print(type(tensor3))
print(type(tf_constant))
print("----")
print(tensor3)
print(tf_constant)

<class 'list'>
<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
----
Tensor("Const_12:0", shape=(2, 2), dtype=float32)
Tensor("Const_12:0", shape=(2, 2), dtype=float32)


As you can see from the print statements above, each Tensor object has a unique __name__, __shape__, and __dtype__. Because we did not explicitly provide a name, tensorflow automatically generated names for us (e.g. "Const_12_0"). <br><br>
Tensorflow has a few convenient constructors for initialising frequently used tensors such as: <br>
- `tf.zeros(shape)`
- `tf.ones(shape)`

These can be used with standard numerical operations to create other numbers. For example, we can create a 500x500 tensor containing of 0.5s:

In [10]:
halves = tf.ones([500, 500]) * 0.5

### 2.3 Creating Operators ###
Tensorflow comes with its own operators which take tensors as their arguments. One of the simplest ones is the `tf.negative(tensor)` which takes a tensor as input and returns another tensor with every element of the input negated:


In [14]:
x = tf.constant([1.0,2.0])
x_neg = tf.negative(x)

Here is a list of other commonly used operators in tensorflow:
- `tf.add(x, y)`
- `tf.substract(x, y)`
- `tf.multiply(x, y)` (Note: element-wise)
- `tf.div(x, y)`
- `tf.floordiv(x, y)`
- `tf.truediv(x, y)` (Casts the arguments as float)
- `tf.mod(x, y)` Element wise remainder after dividing x by y
- `tf.pow(x, y)` Element wise power of x to y
- `tf.exp(x)`
- `tf.sqrt(x)`

<a href="https://www.tensorflow.org/api_guides/python/math_ops">Here</a> you can find the complete list of operators available in the documentation. Using the above operators, we can now write the gaussian pdf using tensors:

In [17]:
mean = 0.0
sigma = 1.0

norm_tensor = tf.exp(tf.negative(tf.pow(x - mean, 2.0)/(2.0 * tf.pow(sigma,2.0) ))) * (1.0 / (sigma * tf.sqrt(2.0 * pi) ))

### 2.4 Executing operators with sessions ###
Notice that until now, we have simply defined the behaviour of tensors - no calculations have yet been performed. For calculations to be carried out, a __session__ is required. To create a session we use the `tf.session()` function:

In [20]:
x = tf.constant([1., 2., 3.], dtype=tf.float32)
neg_x = tf.negative(x)

with tf.Session() as sess:
    result = sess.run(neg_x)

print(result)

[-1. -2. -3.]


Every Tensor object has an `eval()` function to evaluate the mathematical operations that defines its value. However, the `eval()` function requires defining a session object for the library to understand how best to make use of hardware. When we use `sess.run(...)` it is equivalent to invoking the Tensor's `eval()` function within the context of the created session.<br>

Sometimes, for presentation purposes it is more convenient to create an __interactive session__ where the session is implicit in any call to `eval()`. It is important to remember to `close()` the interactive mode to free up the taken resources:

In [21]:
sess = tf.InteractiveSession()

x = tf.constant([1., 2., 3.], dtype=tf.float32)
mean = 0.0
sigma = 1.0
norm_tensor = tf.exp(tf.negative(tf.pow(x - mean, 2.0)/(2.0 * tf.pow(sigma,2.0) ))) * (1.0 / (sigma * tf.sqrt(2.0 * pi) ))

result = norm_tensor.eval()
print(result)

sess.close()

[ 0.24197073  0.05399097  0.00443185]


In [3]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    result = sess.run(neg_x)

print(result)

[-1. -2.]


In [8]:
sess = tf.InteractiveSession()
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spike = tf.Variable(False)
spike.initializer.run()

In [9]:
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        updater = tf.assign(spike, True)
        updater.eval()
    else:
        tf.assign(spike, False).eval()
    
    print("Spike: %s" % spike.eval())

Spike: False
Spike: True
Spike: False
Spike: False
Spike: True
Spike: False
Spike: True


In [10]:
sess.close()

In [22]:
# Suppose we want to save results so that they are accessible without rerunning the whole model.
# Tensorflow offers us a simple way to do this:

sess = tf.InteractiveSession()
 
raw_data = [1., 2., 8., -1., 0., 5.5, 6., 13]
spikes = tf.Variable([False] * len(raw_data), name='spikes')
spikes.initializer.run()
 
saver = tf.train.Saver()
 
for i in range(1, len(raw_data)):
    if raw_data[i] - raw_data[i-1] > 5:
        spikes_val = spikes.eval()
        spikes_val[i] = True
        updater = tf.assign(spikes, spikes_val)
        updater.eval()
 
 
save_path = saver.save(sess, "./trained_variables.ckpt")
# print("spikes data saved in file: %s" % save_path)
 
sess.close()
